# IBM Auto AI RAG 사용하기
IBM Auto AI RAG를 사용하여 RAG를 만들고 배포한 AI 서비스를 LangChain과 결합하여 최종 응답을 처리합니다.
이 노트북은 IBM Auto AI를 통해 구축된 RAG(Retrieval-Augmented Generation) 시스템을 LangChain과 통합하여 AI 애플리케이션을 구축하는 방법을 안내합니다.

# 라이브러리 설치하기
Lab에서 Auto AI RAG를 사용하기 위해서는 아래의 라이브러리를 설치해야 합니다.
이 셀에서는 필요한 라이브러리들을 설치합니다. `langchain`, `langchain_openai`, `langchain_community`, 그리고 `langchain-ibm`을 설치합니다.  `TOKENIZERS_PARALLELISM` 환경 변수를 설정하여 병렬 처리 관련 경고를 억제합니다.

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

!pip install langchain==0.3.0
!pip install langchain --upgrade

!pip install langchain_openai
!pip install langchain_community

!pip install -U langchain-ibm
!pip install ibm-watsonx-ai==1.3.6

라이브러리 설치가 완료 되었으면 노트북 커널을 Restart 합니다.
메뉴에서 `kernel > restart` 를 실행합니다.
설치된 라이브러리를 적용하기 위해 노트북 커널을 재시작해야 합니다.

# IBM AutoAI RAG 서비스를 적용한 RAG 시스템 구현
이 섹션에서는 배포한 IBM AutoAI RAG 서비스를 파이썬 코드를 사용하여 호출하는 방법을 설명합니다.
이 코드는 IBM Cloud watsonx.ai를 사용하여 IBM AutoAI RAG 서비스를 생성 및 배포한 후에 실행해야 합니다. 이 코드는 IBM AutoAI RAG 서비스의 REST API를 호출하여 모델을 생성하고 예측을 수행하는 방법을 보여줍니다.


In [ ]:
from langchain_core.documents import Document
import json

def convert_to_documents(json_data):
    """
    JSON 데이터를 LangChain Document 객체로 변환하는 함수.
    
    Args:
        json_data (str or dict): JSON 문자열 또는 파싱된 JSON 객체
        
    Returns:
        list: LangChain Document 객체의 리스트
    """
    # JSON 데이터가 문자열인 경우 파싱
    if isinstance(json_data, str):
        try:
            data = json.loads(json_data)
        except json.JSONDecodeError as e:
            raise ValueError(f"JSON 파싱 오류: {e}")
    else:
        data = json_data

    # Document 객체를 저장할 리스트
    documents = []

    # JSON 데이터에서 choices를 추출
    choices = data.get("choices", [])
    if not choices:
        raise ValueError("JSON 데이터에 'choices' 키가 없습니다.")

    # 각 choice에서 reference_documents를 추출
    for choice in choices:
        reference_docs = choice.get("reference_documents", [])
        for ref_doc in reference_docs:
            # page_content와 metadata 추출
            page_content = ref_doc.get("page_content", "")
            metadata = ref_doc.get("metadata", {})

            # Document 객체 생성
            doc = Document(
                page_content=page_content,
                metadata=metadata
            )
            documents.append(doc)

    if not documents:
        raise ValueError("변환된 Document 객체가 없습니다. JSON 데이터에 'reference_documents'가 포함되어 있는지 확인하세요.")

    return documents

"CLOUD_API_KEY"는 IBM Cloud의 watsonx 서비스에서 API KEY 정보를 가져옵니다.   
"MY_PROJECT_ID"는 IBM Cloud의 watsonx 서비스에서 프로젝트 ID 정보를 가져옵니다.


In [ ]:
# IBM Cloud API Key와 URL 설정
api_key = "<CLOUD_API_KEY>"
# region에 따라 주소가 다를 수 있습니다. 주소를 확인해 주세요.
ibm_cloud_url = "https://us-south.ml.cloud.ibm.com" 
project_id = "<MY_PROJECT_ID>"

# 배포한 AutoAI RAG 모델의 URL
auto_ai_rag_url = "<IBM_AUTOAI_RAG_URL>"
auto_ai_rag_stream_url = "<IBM_AUTOAI_RAG_STREAM_URL>"

이 셀에서는 사용자 입력을 설정합니다. `user_input` 변수에 질문을 할당합니다.

In [ ]:
user_input = '기업의 생성형AI 도입 현황은?'
# user_input = "기업에서 생성형 AI를 도입하는 가치는?"
# user_input = "기업에서 생성형 AI를 도입하기 위해 고려사항은?"
# user_input = "기업에서 생성형 AI 도입을 가로막는 가장 큰 장애물은?"

이 셀에서는 IBM Cloud API를 사용하여 인증 토큰을 가져오고, AutoAI RAG 서비스에 요청을 보내 응답을 스트리밍 방식으로 처리합니다.

In [ ]:
import requests
import json

# IBM Cloud API Key를 사용하여 토큰 발급
token_response = requests.post(
    'https://iam.cloud.ibm.com/identity/token',
    data={
        "apikey": api_key,
        "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'
    }
)
mltoken = token_response.json().get("access_token")
if not mltoken:
    raise Exception("토큰 발급 실패!")

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {mltoken}',
    'Accept': 'text/event-stream'  # 스트리밍 응답 처리용 헤더
}

payload_scoring = {
    "messages": [
        {
            "role": "user",
            "content": user_input
        }
    ]
}

response = requests.post(
    auto_ai_rag_url,
    headers=headers,
    json=payload_scoring,
    stream=True
)


print("=== 응답 결과 ===")
print(response.json())

# IBM AutoAI RAG 서비스와 Rangchain을 통합한 시스템 구현

LangChain은 다양한 데이터 소스와 AI 모델을 연결하여 Retrieval-Augmented Generation(RAG) 시스템을 구축하는 데 유용한 라이브러리입니다. RAG는 검색 기반 접근 방식을 활용하여 대규모 언어 모델(LLM)의 성능을 향상시키는 기술입니다. 

여기서는 IBM AutoAI RAG 서비스의 강력한 데이터 처리 기능과 LangChain의 유연한 체인 구성 기능을 결합하여 RAG 시스템을 구현하는 방법을 다룹니다. 이를 통해 다음과 같은 작업을 수행할 수 있습니다:
- IBM AutoAI RAG 데이터 소스에서 정보를 검색
- 검색된 정보를 기반으로 LLM을 활용하여 질문에 대한 답변 생성
- LangChain을 사용하여 체인을 구성하고, 데이터 소스와 AI 모델 간의 상호작용을 관리

이 시스템은 기업이 생성형 AI를 활용하여 노코드 기반의 AI RAG 서비스를 생성하고, RagChain을 통해 이를 통합하여 다양한 비즈니스 문제를 해결하는 데 도움을 줄 수 있습니다.

## IBM AutoAI RAG 모델을 사용하여 문서 검색
IBM watsonx.ai에 배포된 AutoAI 기반 RAG 서비스를 활용하여 문서 검색과 질의 응답을 수행합니다. IBM에서 제공하는 SDK를 이용해 AI 서비스를 호출하며, 사용자의 질문을 전달하면 AutoAI RAG 서비스는 답변과 함께 관련 문서 목록도 함께 반환합니다. 이를 위해  ```call_ai_service``` 함수를 정의하여 AI 서비스 호출, 응답 처리, 관련 문서 추출을 일괄적으로 수행합니다.

In [ ]:
import requests
import json

def call_ai_service (user_input):

    # IBM Cloud API Key를 사용하여 토큰 발급
    token_response = requests.post(
        'https://iam.cloud.ibm.com/identity/token',
        data={
            "apikey": api_key,
            "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'
        }
    )
    mltoken = token_response.json().get("access_token")
    if not mltoken:
        raise Exception("토큰 발급 실패!")
    
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {mltoken}',
        'Accept': 'text/event-stream'  # 스트리밍 응답 처리용 헤더
    }
    
    payload_scoring = {
        "messages": [
            {
                "role": "user",
                "content": user_input
            }
        ]
    }
    
    response = requests.post(
        auto_ai_rag_url,
        headers=headers,
        json=payload_scoring,
        stream=True
    )
    return response

## IBM watsonx.ai 모델 설정
watsionx.ai는 IBM의 AI 플랫폼으로, 다양한 AI 모델과 서비스를 제공합니다. 아래 코드는 watsonx.ai 모델을 사용할 수 있도록 watsonx.ai의 AI 모델에 연결합니다.
* 모델 파라미터 설정
* 모델 이름 설정

In [ ]:
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain_ibm import WatsonxLLM


import os
os.environ["WATSONX_URL"] = ibm_cloud_url
os.environ["WATSONX_APIKEY"] = api_key
# os.environ["WATSONX_TOKEN"] = ""

# 모델 파라미터 설정
params = {
    "decoding_method": "greedy",
    "temperature": 0.1,
    "min_new_tokens": 50,
    "max_new_tokens": 1000
}

# WatsonxLLM 모델 초기화
model_llm = WatsonxLLM(
    model_id="meta-llama/llama-3-3-70b-instruct",  # 모델 이름 설정
    project_id=project_id,
    params=params
)

## LangChain을 사용하여 AI 서비스와 통합
LangChain을 사용하여 IBM AutoAI RAG 서비스와 통합하여 RAG 시스템을 구축합니다. LangChain의 `create_stuff_documents_chain` 체인을 사용하여 검색 기반 질의 응답 시스템을 구현합니다. 이 체인은 검색기와 LLM을 결합하여 사용자의 질문에 대한 답변을 생성합니다.

배포한 Auto RAG AI 서비스에서 데이터를 검색 후 Langchain과 연동하기 위해 데이타 타입을 Document 형식으로 변환합니다.

In [ ]:
user_input = '기업의 생성형AI 도입 현황은?'
# user_input = "기업에서 생성형 AI를 도입하는 가치는?"
# user_input = "기업에서 생성형 AI를 도입하기 위해 고려사항은?"
# user_input = "기업에서 생성형 AI 도입을 가로막는 가장 큰 장애물은?"


reference_docs = call_ai_service(user_input)
# 변환된 문서 리스트
docs_search = convert_to_documents(reference_docs.json())

watsonx에 배포한 AI 서비스의 응답 결과를 Langchain과 결합하여 RAG 시스템을 구축할 수 있습니다. Langchain은 다양한 데이터 소스와 AI 모델을 연결하여 RAG 시스템을 구축하는 데 유용한 라이브러리입니다. Langchain을 사용하면 데이터 소스에서 정보를 검색하고, AI 모델을 사용하여 새로운 텍스트를 생성하는 작업을 쉽게 수행할 수 있습니다.

In [ ]:
# 프롬프트 정의
prompt = PromptTemplate.from_template("""
아래 문서를 참고해서 사용자 질문에 정확하게 한국어로 답변해줘.  
- 주어진 질문에 대한 답변만 제공하고, 추가적인 질문이나 답변은 절대 생성하지 마.  
- 답변은 간결하게 3문장 이내로 작성하며, 문서에 기반한 사실만 포함해.

문서:
{context}

질문: {input}

답변:
""")

# 문서 체인 구성
stuff_chain = create_stuff_documents_chain(model_llm, prompt)


# 실행
result = stuff_chain.invoke({"context": docs_search, "input":user_input})
print(result)